# Mean and Standard Deviation

Why do we need standard deviation? We will demonstrate its usefulness by studying a temperature dataset. 

## The Temperature Dataset

This dataset covers a variety of cities in the United States across all States and Territories. For each city, we have the average temperature in each month. The unit of observation is at the city/month level. We have variables for the state, the city, the month and the average temperature.

The dataset, *TempCitiesUSA.csv*, can be downloaded [here](../data/TempCitiesUSA.csv).

In [41]:
# Load in Data Tools
# For Reading/Loading Data
library(readr)
library(tibble)
library(dplyr)
library(ggplot2)
# Load in Data
df_temp <- read_csv('../data/TempCitiesUSA.csv')

Parsed with column specification:
cols(
  state = col_character(),
  city = col_character(),
  month = col_double(),
  temp.f = col_double()
)


## Listing Variables in the Dataset

We can see that the state and city variables are string variables. We can show unique states and cities by months.

In [45]:
# A function that shows Unique Values for Categorical Variables in a Table format
show.unique.values <- function(df, cate.var.str){
    
    # Unique Categories
    unique.cates <- unique(df$[[cate.var.str]])

    # At most 10 columns
    unique.count <- length(unique.cates)
    col.count <- min(ceiling(sqrt(unique.count)), 10)
    row.count <- ceiling(unique.count/col.count)

    # Reshape
    expand.length = row.count*col.count
    unique.cates.expand <- vector(mode = "character", length = expand.length)
    unique.cates.expand[0:unique.count] <- unique.cates
    dim(unique.cates.expand) <- c(row.count, col.count)

    # Show
    as.tibble(unique.cates.expand, rownames = NA)
}

ERROR: Error in parse(text = x, srcfile = src): <text>:5:31: unexpected '('
4:     # Unique Categories
5:     unique.cates <- unique(df$(
                                 ^


In [43]:
# List of categorical Variables
cate.vars.list <- c('state', 'city', 'month')
lapply(cate.vars.list, show.unique.values, df_temp)

ERROR: Error in df[[cate.var.str]]: attempt to select more than one element in vectorIndex


# Data Examples

## College Education Share and Hourly Wage

Two variables:

1. Fraction of individual with college degree in a state
    + this is in Fraction units, the minimum is 0.00, the maximum is 100 percent, which is 1.00
2. Average hourly salary in the state
    + this is in Dollar units

In [ ]:
# Load in Data Tools
# For Reading/Loading Data
library(readr)
library(tibble)
library(dplyr)
library(ggplot2)
# Load in Data
df_wgedu <- read_csv('../data/EPIStateEduWage2017.csv')

## A Scatter Plot

We can Visualize the Data with a Scatter Plot. There seems to be a positive relationship between the share of individuals in a state with college education, and the average hourly salary in that state. 

While most states are along the trend line, we have some states, likw WY, that are outliers. WY has high hourly salary but low share with college education.

In [ ]:
# Control Graph Size
options(repr.plot.width = 5, repr.plot.height = 5)
# Draw Scatter Plot
# 1. specify x and y
# 2. label each state
# 3. add in trend line
suppressMessages(print(
    ggplot(df_wgedu, aes(x=Share.College.Edu, y=Hourly.Salary)) +
      geom_point(size=1) +
      geom_text(aes(label=State), size=3, hjust=-.2, vjust=-.2) +
      geom_smooth(method=lm) +
      labs(title = 'Hourly Wage and College Share by States',
           x = 'Fraction with College Education',
           y = 'Hourly Wage',
           caption = 'Economic Policy Institute\n www.epi.org/data/') +
      theme_bw()
))

## Standard Deviations and Coefficient of Variation

The two variables above are in different units. We first calculate mean, standard deviation, and covariance. With just these, it is hard to compare the standard deviation of the two variables, which are in different scales. 

The sample standard deviations for the two variables are: $0.051$ and $1.51$, in fraction and dollar units. Can we say the hourly salary has a larger standard deviation? But it is just a different scale. $1.51$ is a large number, but that does not mean that variable has greater variation than the fraction with college education variable. 

Converting the Statistics to Coefficient of Variations, now we have: $0.16$ and $0.09$. Because of the division, these are both in fraction units--standard deviations as a fraction of the mean. Now these are more comparable.

In [ ]:
# We can compute the three basic statistics
stats.msdv <- list(
              # Mean, SD and Var for the College Share variable
              Shr.Coll.Mean = mean(df_wgedu$Share.College.Edu), 
              Shr.Coll.Std = sd(df_wgedu$Share.College.Edu),
              Shr.Coll.Var = var(df_wgedu$Share.College.Edu),
    
              # Mean, SD and Var for the Hourly Wage Variable
              Hr.Wage.Mean = mean(df_wgedu$Hourly.Salary),                            
              Hr.Wage.Std = sd(df_wgedu$Hourly.Salary),
              Hr.Wage.Var = var(df_wgedu$Hourly.Salary)
              )

# We can compute the three basic statistics
stats.coefvari <- list(              
              # Coefficient of Variation
              Shr.Coll.Coef.Variation = (stats.msdv$Shr.Coll.Std)/(stats.msdv$Shr.Coll.Mean),
              Hr.Wage.Coef.Variation = (stats.msdv$Hr.Wage.Std)/(stats.msdv$Hr.Wage.Mean)
              )

# Let's Print the Statistics we Computed
as_tibble(stats.msdv)
as_tibble(stats.coefvari)

## Covariance and Correlation

For covariance, hard to tell whether it is large or small. To make comparisons possible, we calculate coefficient of variations and correlation statistics.

The covariance we get is positive: $0.06$, but is this actually large positive relationship? $0.06$ seems like a small number. 

Rescaling covariance to correlation, the correlation between the two variables is: $0.78$. Since the correlation of two variable is beloww $-1$ and $+1$, we can now say actually the two variables are very positively related. Higher share of individuals with college education is strongly positively correlated with higher hourly salary. 

In [ ]:
# We can compute the three basic statistics
states.covcor <- list(              
              # Covariance between the two variables
              Shr.Wage.Cov = cov(df_wgedu$Hourly.Salary,
                                 df_wgedu$Share.College.Edu),        
              # Correlation 
              Shr.Wage.Cor = cor(df_survey$Hourly.Salary, df_survey$Share.College.Edu),
              Shr.Wage.Cor.Formula = (cov(df_wgedu$Hourly.Salary, df_wgedu$Share.College.Edu)
                                      /(stats.msdv$Shr.Coll.Std*stats.msdv$Hr.Wage.Std))
              )

# Let's Print the Statistics we Computed
as_tibble(states.covcor)